In [1]:
from pathlib import Path
import json

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import googlemaps
import polyline
import requests


DATA_DIR = Path('../data')
OUT_DIR = Path('../output')

In [2]:
path = DATA_DIR/'wellington_gtfs_20171016.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.assess_quality()

,indicator,value
0,num_route_short_names_duplicated,0
1,frac_route_short_names_duplicated,0
2,num_stop_time_dists_missing,340007
3,frac_stop_time_dists_missing,1
4,num_direction_ids_missing,0
5,frac_direction_ids_missing,0
6,num_trips_missing_shapes,0
7,frac_trips_missing_shapes,0
8,num_departure_times_missing,0
9,frac_departure_times_missing,0


In [3]:
# shapes_g = feed.shapes_to_geojson()
# path = Path('../wellington_shapes_20171016.geojson')
# with path.open('w') as tgt:
#     json.dump(shapes_g, tgt)

In [4]:
"""
If no shapes, use stops only.
If shapes, then add distances to stop times and to shapes
"""
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)


In [5]:
def refine(xs, n):
    """
    Given a strictly increasing NumPy array ``xs`` of at least two numbers
    x_1 < x_2 < ... < x_r and a nonnegative integer ``n``, 
    insert into the list ``n`` more numbers between x_1 and x_r
    in a spread-out way.
    Return the resulting list as a NumPy array.
    """
    while n > 0:
        diffs = np.diff(xs)

        # Get indices i, j of biggest diffs d_i > d_j.
        # Use the method at https://stackoverflow.com/a/23734295 for speed.
        try:
            indices = np.argpartition(diffs, -2)[-2:]
            i, j = indices[np.argsort(diffs[indices])[::-1]]
            d_i, d_j = diffs[i], diffs[j]
            
            # Choose k => 1 least such that d_i/(k + 1) < d_j
            # with the intent of inserting k evenly spaced points 
            # between x_i and x_{i+1}
            k = int(max(1, np.ceil(d_i/d_j - 1)))

            # Shrink k if necessary so as not to exceed number of remaining points
            k = min(k, n)
        except ValueError:
            # Here xs has only two elements, hence diffs has only one element.
            # Using try-except because faster than if-else.
            i = 0
            d_i = diffs[0]
            k = n
        
        # Insert the k points, updating xs
        xs = np.concatenate([
          xs[:i + 1], 
          [xs[i] + s*d_i/(k + 1) for s in range(1, k + 1)], 
          xs[i + 1:]
          ])
        
        # Update n
        n -= k
        
    return xs

In [6]:
# Test refine() some

xs = np.array([0, 3/4, 1])
assert np.array_equal(refine(xs, 0), xs)
assert np.array_equal(refine(xs, 1), np.array([0, 3/8, 3/4, 1]))
assert np.array_equal(refine(xs, 2), np.array([0, 1/4, 1/2, 3/4, 1]))
assert np.array_equal(refine(xs, 3), np.array([0, 1/4, 1/2, 3/4, 7/8, 1]))

In [7]:
def get_stop_patterns(feed, trip_ids=None, sep='-'):
    """
    Return the DataFrame ``feed.trips`` with the additional column
    
    - ``'stop_pattern'``: string; the stop IDs along the 
      trip joined by the separator ``sep``
      
    If a list of trip IDs is also given, then restrict the output
    to those trip IDs.
    """
    st = feed.stop_times.sort_values(['trip_id', 'stop_sequence'])
    if trip_ids is not None:
        # Filter to given trip IDs
        st = st[st['trip_id'].isin(trip_ids)].copy()
            
    def get_pattern(group):
        return group.stop_id.str.cat(sep=sep)
        
    f = st.groupby('trip_id').apply(get_pattern).reset_index().rename(
      columns={0: 'stop_pattern'})
    return feed.trips.merge(f)


In [8]:
def build_sample_points(feed, trip_ids=None, n=100):
    """
    Given a GTFS feed (GTFSTK Feed instance), 
    preferably with a ``feed.stop_times.shape_dist_traveled`` column, 
    return a dictionary of the form 
    
    stop pattern -> list of n (longitude, latitude) sample points along route.
    
    Regarding the list of sample points, suppose a stop pattern
    witnessed by some trip has k stops.
    If k < n, the trip has a shape, and all the ``shape_dist_traveled``
    values of the trip's stop times are present, then the sample points 
    comprise the k stops of the trip along with ``n - k`` additional points
    somewhat evenly sampled from the trip's shape, all in the order of the trip's travel.
    Else if k > n, then the sample points include only n stops: no points (n=0); 
    the first stop (n=1); the first and the last stop (n=2);
    the first, last, and n - k random stops (n > 2).
    Else, the sample points are the k stops.
    
    If a list of trip IDs is given, then restrict to the stop patterns
    of those trips.
    Otherwise, build sample points for every stop pattern.
    
    NOTES:
    
    - In the case of choosing random stops, the choices will be the same
      for across all runs of this function (by using a fixed 
      random number generator seed), which is good for debugging.
    - The implementation assumes that if two trips have the same stop pattern, 
      then they also have the same shape.
    """
    # Seed random number generator for reproducible results
    np.random.seed(42)
    
    if trip_ids is None:
        # Use all trip IDs
        trip_ids = feed.trips.trip_id
    
    # Get stop patterns and choose a representative trip for each one
    t = get_stop_patterns(feed, trip_ids)
    t = t.groupby('stop_pattern').agg('first').reset_index()
    trip_ids = t.trip_id
    
    # Get stops times for the representative trips
    st = feed.stop_times
    st = st[st['trip_id'].isin(trip_ids)].sort_values(
      ['trip_id', 'stop_sequence'])
    
    # Join in stop patterns and shapes
    st = st.merge(t[['trip_id', 'shape_id', 'stop_pattern']])
    
    # Join in stop locations
    st = st.merge(feed.stops[['stop_id', 'stop_lon', 'stop_lat']])

    # Create shape_dist_traveled column if it does not exist
    if not 'shape_dist_traveled' in st:
        st['shape_dist_traveled'] = np.nan
    
    # Get shape geometries
    geom_by_shape = feed.build_geometry_by_shape(shape_ids=t.shape_id) or {}

    # Build dict stop pattern -> list of (lon, lat) sample points.
    # Since t contains unique stop patterns, no computations will be repeated.
    points_by_sp = {}    
    for stop_pattern, group in st.groupby('stop_pattern'):
        shape_id, stop_pattern = group[['shape_id', 'stop_pattern']].iloc[0]           
        k = group.shape[0]  # Number of stops along trip
        if k < n and (shape_id in geom_by_shape)\
          and group['shape_dist_traveled'].notnull().all():
            # Start with stop points, and mark their distances for later sorting.
            # Scale distances to interval [0, 1] to avoid changing coordinate systems.            
            group['shape_dist_traveled'] /= group['shape_dist_traveled'].max()
            stop_points = group[['stop_lon', 'stop_lat', 'shape_dist_traveled']].values
            dists = group['shape_dist_traveled'].values
            
            # Get n - k nicely spaced points from trip shape.
            new_dists = np.setdiff1d(refine(dists, n - k), dists)                 
            geom = geom_by_shape[shape_id]
            shape_points = [
              list(geom.interpolate(d, normalized=True).coords[0]) + [d]
              for d in new_dists]
            
            # Combine with stop points and sort
            points = sorted(np.concatenate([stop_points, shape_points]).tolist(), 
              key=lambda x: x[2])
            
            # Remove distance markers
            points = [x[:2] for x in points]

        elif k > n:
            # Use n stop points only
            if n == 0:
                points = []
            elif n == 1:
                # First stop
                points = group[['stop_lon', 'stop_lat']].iloc[0].values.tolist() 
            elif n == 2:
                # First and last stop
                ix = [0, k - 1]
                points = group[['stop_lon', 'stop_lat']].iloc[ix].values.tolist() 
            else:
                # First, last, and n - 2 random stops
                ix = np.concatenate([[0, k - 1], 
                  np.random.choice(range(1, k - 1), n - 2, replace=False)])
                ix = sorted(ix)
                points = group[['stop_lon', 'stop_lat']].iloc[ix].values.tolist()
        else:
            # Best can do is use the stop points
            points = group[['stop_lon', 'stop_lat']].values.tolist()

        points_by_sp[stop_pattern] = points

    return points_by_sp

In [9]:
def get_secret(key, path='../secrets.json'):
    """
    """
    with Path(path).open() as src:
        secrets = json.load(src)
    return secrets[key]


In [10]:
def encode_points_google(points):
    """
    Given a list of longitude-latitude points, return their string
    representation suitable for Google's Snap to Roads API;
    see https://developers.google.com/maps/documentation/roads/snap.
    """
    return ('|').join(['{:.06f},{:.06f}'.format(p[1], p[0]) for p in points])

def decode_points_google(points):
    """
    Inverse of function :func:`encode_points_google`.
    """
    return [[float(x) for x in p.split(',')[::-1]] for p in points.split('|')]

def parse_response_google(response):
    return [[x['location']['longitude'], x['location']['latitude']] for x in response]

def query_google(points, api_key):
    gmaps = googlemaps.Client(api_key)
    points_g = encode_points_google(points)
    response = gmaps.snap_to_roads(points_g, interpolate=True)
    return parse_response_google(response)

points = [
    [174.7963112, -41.25589183999999],
    [174.7836823, -41.27428288],
    [174.7873955, -41.24846454],
]
encode(points)
decode(encode(points))


NameError: name 'encode' is not defined

In [69]:
def encode_points_mapzen(points):
    """
    Given a list of longitude-latitude points, return their dictionary
    representation suitable for Mapzen's Map Matching API;
    see https://mapzen.com/documentation/mobility/map-matching/api-reference/#example-trace_attributes-requests
    """
    return [{'lon': round(p[0], 6), 'lat': round(p[1], 6)} for p in points]

def decode_points_mapzen(points):
    """
    Inverse of function :func:`encode_points_mapzen`.
    """
    return [[d['lon'], d['lat']] for d in points]

def parse_response_mapzen(response):
    pline = polyline.decode(response['trip']['legs'][0]['shape'])
    
    # Need to divide coordinates by 10 for some strange reason
    return [(p[1]/10, p[0]/10) for p in pline]

def query_mapzen(points, api_key, url='https://valhalla.mapzen.com/trace_route',
  kwargs=None):
    data = {
        'shape': encode_points_mapzen(points),
        'costing': 'auto',
    }
    if kwargs is not None:
        data.update(kwargs)

    r = requests.post(url, params={'api_key': api_key}, json=data)
    print(r.json())
    r.raise_for_status()
    return parse_response_mapzen(r.json())


points = [
    [174.7963112, -41.25589183999999],
    [174.7836823, -41.27428288],
    [174.7873955, -41.24846454],
]
# points = [
#     [-76.983345, 38.892459],
#     [-76.983353, 38.892457],
#     [-76.98336, 38.892457],
#     [-76.983365, 38.892458],
#     [-76.983369, 38.892454]
# ]
query_mapzen(points, get_secret('MAPZEN_API_KEY'))


{'error': 'No path could be found for input', 'status_code': 400, 'status': 'Bad Request', 'error_code': 442}


HTTPError: 400 Client Error: Bad Request for url: https://valhalla.mapzen.com/trace_route?api_key=mapzen-jLrDBSP

In [62]:
t = feed.trips.merge(feed.routes)
t = t[t['route_type'] == 3].sample(frac=0.01)
%time sample_points_1 = build_sample_points(feed, t.trip_id, 4)
%time sample_points_2 = build_sample_points(feed, t.trip_id, 100)


CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 196 ms


/home/araichev/.virtualenvs/snap_bus_routes/lib/python3.5/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 2.17 s, sys: 4 ms, total: 2.17 s
Wall time: 2.17 s


In [70]:
sp = list(sample_points_1.keys())[0]
print('num stops=', len(sp.split('-')))
points_1 = sample_points_1[sp]
points_2 = sample_points_2[sp]

num stops= 49


In [71]:

l2 = sg.LineString(points_2)
points_2s = query_mapzen(points_2, get_secret('MAPZEN_API_KEY'))
l2s = sg.LineString(points_2s)

# Get actual trip shape
p = get_stop_patterns(feed, t.trip_id)
shid = p.loc[p['stop_pattern'] == sp, 'shape_id'].unique()[0]
l3 = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]


{'trip': {'legs': [{'shape': 'peglmA}akmlIkBiv@kAiLyGs[{AqGs@iMTmJ~Hs[dUey@vCoIfJiMvHwClYyBhHwCdFsFfCqHl@sF?kLyBeOyGuY{AwDtJkKbG_TzEoHl@_@hH{@jk@tDtJtE|D?jF{@fDsGd@oHMcQsBgNkFgNiCsGyAyB}EwCm_@yW~CkK~DuYjAaSOePaCaQsAyC{FkK_IqH}NqGkBpGgCtEoDtEkQ`RyQbQqBzAsBzAyB?aC\\kU{AyB\\aC^}d@|SaG|@yG{AoIwC_D_@aI?sZxCcLgNaBwDaH{UW]sAyB}N}TiCqHu_@ukBaHyWoCoI}EoIyBwC|_@mt@bAyAfDqHfDuExGsFdEuDdF{Bt^kKbB]vpAo]|E}@fI?vpAdP|YtDriAlKhWvC~I?`G?vTyBhQwCtPkLfHqGbGqHdKyWnHyVtEwX|O_rAbz@bf@`jA|h@x`Azj@|hA|i@xWjKxVrGfr@lJrtAbPvXvDpf@pGdlB|Txp@nIrp@zAvR]|PyCz@{@bA}@|EcPlE{sAl@cQTmJm@?e@}@e@]E}@G{@F}@d@{@d@_@r@]V\\l@?d@^Tz@`C{@hB_@rG?hG^fJ_@rE]lF{@tDyCjGmI`CuFz@wCe@]]}@G{@F}@L{@d@]|@}@t@?l@\\\\^hLyWtJ{Us@{AG{AF{AT]l@{@bA}@dA?jA|@l@z@TzAGzAUz@vIpHxFtD~DzAhG\\l~@{A`W{@lF\\?{A\\yAl@}@r@]l@?^uEj@uElAoIdOyk@\\uDN{ANyBDyBFyBM]G}@F{@T_@T]d@]bByB`CyC`CuDhByBbB{AxByBnC{A`C{@fD_@|D]tE]rA?zA]|D}@bB]bGyBbVaSbFyArF_@hC?vS^fXz@t@?FmJLeOl@uE|@{AlEyBnC?b`A]zxA}@z@^bAz@^xBLzA^bQ_@vCkAvCyBzAgDzAyB}@mDyAwD_@a]|@so@]kF\\wDz@FxBU|@_@z@k

In [73]:
# Plot
import folium


mappy = folium.Map(location=points_2[0][::-1], zoom_start=15)
folium.GeoJson(sg.mapping(l2), style_function=lambda x: {'color': 'blue'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2s), style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

In [85]:
get_stop_patterns(feed)['stop_pattern'].nunique()

468